In [36]:
import re
import nltk
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import pytz
import datetime
import seaborn as sns
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
from tqdm import tqdm
import boto3
import csv

In [53]:
bucket = 'mads-capstone-2023' 
data_key = 'FINRA_Short_Sale_Data2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

In [54]:
#pd.read_csv(obj["Body"].read(), nrows=10)

cnt = 0

for i, row in enumerate(obj["Body"].iter_lines()):
    cnt += 1
    
cnt 

In [55]:
total_records = 8219426

In [59]:
bucket = 'mads-capstone-2023' 
data_key = 'FINRA_Short_Sale_Data2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

for i, row in enumerate(obj["Body"].iter_lines()):
    if i==0:
        continue
        
    row_data = row.decode().split('"')
    row_data =  row_data[0].split(",") + [row_data[1]] + [row_data[2]]
    row_data = [val for val in row_data if val]

['0', 'A', '2018-08-01', '127212.0', '14.0', '658064.0', 'Q,N', ',0.1933125045588271']
['1', 'AA', '2018-08-01', '514915.0', '817.0', '1549350.0', 'Q,N', ',0.3323425952818924']
['2', 'AABA', '2018-08-01', '975797.0', '370.0', '3608955.0', 'Q,N', ',0.27038214663247395']
['3', 'AAC', '2018-08-01', '43229.0', '0.0', '93929.0', 'Q,N', ',0.4602305997082903']
['4', 'AADR', '2018-08-01', '3213.0', '0.0', '9620.0', 'Q,N', ',0.333991683991684']
['5', 'AAL', '2018-08-01', '1373187.0', '3950.0', '2142375.0', 'Q,N', ',0.6409648170838439']


In [60]:
bucket = 'mads-capstone-2023' 
data_key = 'FINRA_Short_Sale_Data2.csv' 

s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)

In [ ]:
%%time

records = []

for i, row in enumerate(obj["Body"].iter_lines()):
    if i == 0:
        continue
    try:
        row_data = row.decode().split(",")
        row_data =  row_data[0].split(",") + [row_data[1]] + [row_data[2]]
        row_data = [val for val in row_data if val]
        idx, ticker, dt, shortvolume, shortexemptvolume, totalvolume, market, short_ratio  = row_data
        dt = dt.split()[0]
        shortvolume = float(shortvolume) if shortvolume else None
        shortexemptvolume = float(shortexemptvolume) if shortexemptvolume else None
        totalvolume = float(totalvolume) if totalvolume else None
        short_ratio = float(short_ratio) if short_ratio else None
    except Exception as e:
        print(e)
        print(f"{i} - {row_data} Failed Parsing.")
        continue

    records.append([dt,incr,opt_value,ticker,greek,opt_type])
    
    print(records)

    if (i%100_000)==0: ## Batching 100k records and uploading all at once.
        df = pd.DataFrame(records)#.dropna()
        df.to_csv("finra.csv", header=False, index=False)
        with open("finra.csv", "rb") as fp:
            cursor.copy_from(fp, "finra", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

        records = []
        gc.collect();

        if (i%1_000_000) == 0:
            print(f"{i} Records Inserted")

if records:
    print(len(records))
    df = pd.DataFrame(records)#.dropna()
    df.to_csv("finra.csv", header=False, index=False)
    with open("finra.csv", "rb") as fp:
        cursor.copy_from(fp, "finra", sep=",", null="", columns=["exec_date", "increment", "opt_value", "ticker", "greek", "option_type"])

not enough values to unpack (expected 8, got 3)
1 - ['0', 'A', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
2 - ['1', 'AA', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
3 - ['2', 'AABA', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
4 - ['3', 'AAC', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
5 - ['4', 'AADR', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
6 - ['5', 'AAL', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
7 - ['6', 'AAMC', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
8 - ['7', 'AAME', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
9 - ['8', 'AAN', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)
10 - ['9', 'AAOI', '2018-08-01'] Failed Parsing.
not enough values to unpack (expected 8, got 3)